# 03 - Model Training

This notebook trains the predictive model for travel time estimation.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Add src to path
sys.path.append(str(Path('../src').resolve()))
from model import load_training_data, train_model, save_model

# Load training data
data_path = Path('../data/processed/training_data.csv')
X, y = load_training_data(data_path)

# Train model
model, metrics = train_model(X, y)
print("Model Metrics:", metrics)

# Save model
model_path = Path('../models/travel_time_model.pkl')
save_model(model, model_path)
